# Tutorial

Use data from [ERA5-Land](https://www.ecmwf.int/en/era5-land) to examine time scales of variability in surface variables at the grid point closest to HU Beltsville.

We demonstrate some of the capabilities of [xarray](http://xarray.pydata.org/en/stable/) and other scientific Python packages.

This tutorial notebook is part of [zmoon92/hu-pbl-workshop-2020/tree/master/python-tutorial](https://github.com/zmoon92/hu-pbl-workshop-2020/tree/master/python-tutorial).

In [ ]:
import calendar

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from ipywidgets import interact, Dropdown
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pvlib
from scipy.fftpack import rfft, irfft
import seaborn as sns
import statsmodels.api as sm
import xarray as xr

In [ ]:
%matplotlib notebook

plt.rcParams.update({
    "figure.autolayout": True,
    "axes.xmargin": 0,
})

# define some defaults
xrp = {
    "size": 5.2,  # height; if we pass it to xarray plot methods, it will create a new fig
    "aspect": 1.6,  # aspect * size = width
}
figsize = (xrp["size"]*xrp["aspect"], xrp["size"])  # width, height

## Pre

### Load the data

According to [the GRUAN page](https://www.gruan.org/network/sites/beltsville), we want to find the point closest to 39.0542 °N, 76.8775 °W. In ERA5-Land, which has a resolution of 0.1°, this turns out to be the point (39.1, -76.9). I have already extracted a few years of data for that point to the file `era5-land_bel.nc`. This file is available if you are running the notebook on the MyBinder for the repo (or have cloned the repo). Otherwise, you will need to download it if you want to follow along.

First, we load and examine the data set.

In the notebook, you can interact with the data set's fancy HTML representation, which will show up if you have xarray [v0.15.1+](http://xarray.pydata.org/en/stable/whats-new.html#v0-15-1-23-mar-2020).

In [ ]:
ds = xr.open_dataset("era5-land_bel.nc")
ds

Examine a variable.

In [ ]:
ds.t2m

### Sample time series plots

In [ ]:
ds.u10.plot.line(**xrp);

Notice how xarray does all of the labeling for us, including units and descriptive name for the variable being plotted. This is possible because the data set we have loaded follows the [CF Conventions](https://cfconventions.org/), specifying `units` and `long_name` attributes for each variable.

See the [xarray guide to plotting page](http://xarray.pydata.org/en/stable/plotting.html) for more info.

Using the [`.rolling` method](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.rolling.html) of a data array, we can easily plot moving averages. Here we select the variable `t2m`, the 2-m temperature. See the [xarray indexing/selecting guide page](http://xarray.pydata.org/en/stable/indexing.html) for more info.

In [ ]:
ds.t2m.plot.line(**xrp, lw=0.5, alpha=0.3, label="original hourly data")
ax = plt.gca(); ylabel = ax.get_ylabel()  # save the auto-added label

# Add some moving averages (the time resolution is 1 hour)
ds.t2m.rolling(time=24, center=True).mean().plot(lw=1.0, alpha=0.5, label="1 day moving average")
ds.t2m.rolling(time=24*30, center=True).mean().plot(lw=1.2, alpha=0.8, label="30 day moving average")
ds.t2m.rolling(time=24*90, center=True).mean().plot(lw=1.8, alpha=1.0, label="90 day moving average")
ax.set_ylabel(ylabel)  # put the label back
ax.legend();

👆 It appears that the near-surface temperature data we have could use some quality control: there are quite a few times when temperature spikes to ~310 K during the winter! But we won't worry about that now...

We see that adding more points to our moving average makes the seasonal cycle and interannual variability more clear.

Is there a relationship between the sensible heat flux and near-surface air temperature? Let's make a scatter plot.

In [ ]:
ds.plot.scatter(x="sshf", y="t2m", marker=".", alpha=0.3, edgecolors="none", **xrp);

There doesn't appear to be much of a relationship. How about if we subset the data to only include midday in the summer?

In [ ]:
h = ds.time.dt.hour
mo = ds.time.dt.month
ds.where(
    (h >= 15) & (h <= 19) & (mo >= 6) & (mo <= 8)  # during the summer, we are UTC-4
).plot.scatter(x="sshf", y="t2m", marker=".", alpha=0.5, edgecolors="none", **xrp)
plt.ylim(ymin=286);  # exclude a few outliers

## Tutorial problems

If you are so inclined, pick something in here to work on for a bit.

### Autocorrelation

Examine the autocorrelation plots of the different variables.

You can use, e.g.:
* [`pandas.plotting.autocorrelation_plot`](https://pandas.pydata.org/docs/reference/api/pandas.plotting.autocorrelation_plot.html)
* [`statsmodels.graphics.tsaplots.plot_acf`](https://www.statsmodels.org/stable/generated/statsmodels.graphics.tsaplots.plot_acf.html)

The autocorrelation will be different with daily data. Resample to daily time resolution using the [`.resample`](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.resample.html) method of our data set `ds` and see the difference.

In [ ]:
# import stuff (if necessary)
...

# make an autocorrelation plot or two
...

The strongest correlation between a pair of variables may be at different lags. We visualize this with cross-correlation plots.

[`plt.xcorr`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.xcorr.html) can be used to make such a plot.

Compare the results for hourly data and for data resampled to daily time resolution.

In [ ]:
# pick variables
...

# plot xcorr
...

### Compare variables to Sun position

Which have the strongest correlation?

We can compute Sun position with [pvlib](https://pvlib-python.readthedocs.io/en/stable/). [Astropy](https://www.astropy.org/) could also be used for this.

In [ ]:
sun_pos = pvlib.solarposition.get_solarposition(ds.time.values, ds.latitude.values, ds.longitude.values)
sun_pos.head()

Add some of the Sun position variables to our data set.

In [ ]:
zen_deg = sun_pos["zenith"]
zen = zen_deg.apply(np.deg2rad)
ds["sza"] = ("time", zen_deg, {"long_name": "solar zenith angle", "units": "deg"})
ds["mu"] = ("time", zen.apply(np.cos), {"long_name": "cos(sza)", "units": ""})
ds["selev"] = ("time", sun_pos["elevation"], {"long_name": "solar elevation angle", "units": "deg"})

How does near-surface air temperature compare to Sun angle? 

Note that we select data where the Sun is up.

In [ ]:
day = ds.time.where(ds.selev > 10).dropna("time")

ds.sel(time=day).plot.scatter(x="sza", y="t2m", marker=".", alpha=0.5, edgecolors="none", **xrp);

The correlation appears stronger if we first resample the hourly data to daily. Compare the linear fits, using, e.g., [OLS from statsmodels](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html) to compute them.

In [ ]:
# Find the strength of the linear fit, for hourly and daily data
...

In [ ]:
# Generally explore the data, both Sun position and ERA5 surface variables
...

Follow [this seaborn example](https://seaborn.pydata.org/examples/many_pairwise_correlations.html) to make a pairwise correlation heatmap. 

You can compute the correlation matrix by first converting the data set `ds` to a NumPy array (using `.to_array`) and then using, e.g., `np.corrcoef`, or first converting to a Pandas DataFrame (using `.to_dataframe`) and using the `.corr` method.

In [ ]:
# Make pairwise correlation heatmap
...

### SZA and maximum near-surface temperature

It is well-known that the warmest temperatures generally lag the minimum solar zenith angle (SZA), on both seasonal and hourly (diurnal cycle) time scales.

Find what that lag is.
* seasonal: for each year find the lag between the yearly minimum SZA and maximum daily mean temperatue
* diurnal: estimate time of min SZA and of max T for each day (possibly by interpolating) and examine the distribution of the offsets

In [ ]:
# seasonal: for each year, calculate the two times and substract them
...

In [ ]:
# diurnal
...

### Remove seasonal cycle

This is often done in statistical analyses so that the seasonal cycle doesn't dominate the signal when looking for relationships.

A common method is to represent the seasonal cycle using a number of harmonics, e.g., 10.

Examine the estimated seasonal cycles of different variables and the time series with seasonal cycles removed, using the function provided below to calculate the seasonal cycles. If we have done a good job, we won't see periodic oscillations associated with the seasonal cycle in the anomaly time series.

In [ ]:
# Calculate t_year (here hour-of-year, cf. day-of-year, which the .dt accessors provide) so we can group by it
# I would like to know a better (more elegant) way to do this...
t_year = ds.time - np.r_[[np.datetime64(f"{x}", "D") for x in ds.time.dt.year.values]]  # calc decDOY instead??
ds["t_in_year"] = ("time", t_year, {"long_name": "time-in-year"})
t_year_leap = ds["t_in_year"].where(ds.time.dt.year == 2016, drop=True).drop(["latitude", "longitude"], drop=True)
# note: really we might want to remove leap days in this analysis

In [ ]:
def compute_sc(da, n_harm):
    "Compute the mean seasonal cycle, given an xr.DataArray."
    x = da.groupby(ds.t_in_year).mean().values  # group by hour in year
    z = rfft(x)
    z[n_harm:] = 0
    return xr.DataArray(
        name=f"sc_{da.name}", data=irfft(z), dims="doy", coords=[t_year_leap.dt.days + t_year_leap.dt.seconds / (24*3600)],
        attrs={"long_name": f"mean seasonal cycle in {da.attrs['long_name']}", "units": da.attrs["units"]},
    )

vn = "t2m"  # select variable here
n_harmonics = 2  # observe how changing the number impacts the results

# Compute and plot seasonal cycle
v = ds[vn]
sc = compute_sc(v, n_harmonics)
sc.plot(size=3, aspect=1.5); plt.xlabel("day-of-year")

# Substract seasonal cycle from the variable's time series
year = v.time.dt.year  # also could use the shorthand `...groupby("time.year")...`
v_anom = v.groupby(year).map(lambda x: x - sc[:x.size].values)  # allow for leap years
# Put the attrs back (I think they are lost because sc doesn't have them)
v_anom.attrs.update(dict(long_name=f"{v.attrs['long_name']} anomaly", units=v.attrs["units"]))

# Plot hourly data and daily means
def mu_sig(da):  # (estimated) mean and stdev
    return f"$\mu={da.mean().values:.3g}$\n$\sigma={da.std().values:.3g}$"
v_anom.plot(size=3, aspect=2.5, lw=0.5); plt.title(mu_sig(v_anom), loc="right")
v_anom_daily = v_anom.resample(time="1D").mean(keep_attrs=True)
v_anom_daily.plot(size=3, aspect=2.5, lw=0.5); plt.title(mu_sig(v_anom_daily), loc="right");

## Extras

### Map plot

We plot data from ERA5 (not ERA5-Land) in a 20x20 degree box around the BEL location. The data are mean (over 1979--2019) monthly, thus, they show climatology.

In [ ]:
ds2 = xr.open_dataset("era5_bel-box_months_1979-2019.nc")
ds2

Quickly see what some of the data looks like.

In [ ]:
plt.figure(figsize=(3, 3)); plt.imshow(ds2.t2m.sel(month=1));

Now, plot on map using [Cartopy](https://scitools.org.uk/cartopy/docs/latest/).

In [ ]:
lat = ds2.latitude
ds_subset = ds2.where((lat >= 35) & (lat <= 45), drop=True)  # cut off a bit

fig = plt.figure(figsize=figsize)

def plot_month(variable, imonth):
    # select variable
    da = ds_subset[variable]
    
    # same contour levels for all months. change `40` to get a different number of levels
    l, h = np.floor(da.min().values), np.ceil(da.max().values); step = np.ceil((h-l)/40)
    levels = np.arange(l, h+step, step)
    
    # erase and re-setup
    fig.clear()
    ax = plt.axes(projection=ccrs.Mercator())
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.STATES)  # only US, requires sufficiently recent Cartopy
    gl = ax.gridlines(draw_labels=True)
    #gl.xlabels_top = False; gl.ylabels_right = False  # now deprecated, but switch to this one if the following doesn't work
    gl.top_labels = False; gl.right_labels = False  # preferred syntax
    
    # plot selected month
    data = da.sel(month=imonth)
    data.plot.contourf(x="longitude", y="latitude", levels=levels, ax=ax, transform=ccrs.PlateCarree())
    ax.set_title(f"{ax.get_title()} ({calendar.month_name[imonth]})")

    
interact(plot_month, variable=Dropdown(options=list(ds2.keys())), imonth=(1, 12, 1));

## References

General references:

* <https://xarray.pydata.org/en/stable/>
  - http://xarray.pydata.org/en/stable/related-projects.html#related-projects
* <https://pandas.pydata.org/docs/>